In [1]:
%load_ext autoreload
%autoreload 2

# pd.read_csv is NOT all you need: DataFrame validation in Python

## About Me

- *Data Analyst/Scientist*, or maybe just *Data Tradie* (credit to **Kale Miller** for this term)
- Deals with smol data; from ~5 to 1e7 rows.
  - Once had to use a generator loop in python to parse something too big to fit into memory.
- Training in *Biostatistics* (but no one can prepare you for real world datasets).
- Background in biology, bioinformatics.
- Some training in R
- self-taught pythonista from Udemy, trying pandas before v0.20.0, these meetups, and using it in the real world.

# Why validate anyway?

Real world data is messy and strange!

- Typos.
- Excel magic behaviour.
- No one actually knows about [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601).
- Sometimes numeric rows get cast to string in csv files 🤷‍♀️
- Someone put a text comment in a numeric column
- Your own code introduced something cursed

## The goal is to **avoid complexity in your code**

In [2]:
# import some good data
import numpy as np
import pandas as pd
from PyVal import config

good_data = pd.read_csv(config.good_data, index_col=0)
spicy_data = pd.read_csv(config.spicy_data, index_col=0)
display(good_data.head(2), spicy_data.head(2))
display(good_data.dtypes, spicy_data.dtypes)

,text,int between 0 and 1e6,float between -1 and 1,no nulls allowed,iso 8601 compliant date
0,row0,934944,-0.751395,False,2021-01-01
1,row1,842425,0.113027,True,2021-01-02


,text,int between 0 and 1e6,float between -1 and 1,no nulls allowed,iso 8601 compliant date
0,row0,934944.0,-0.751395,False,2021-01-01
1,row1,842425.0,0.113027,True,2021-01-02


text                        object
int between 0 and 1e6        int64
float between -1 and 1     float64
no nulls allowed              bool
iso 8601 compliant date     object
dtype: object

text                        object
int between 0 and 1e6      float64
float between -1 and 1     float64
no nulls allowed            object
iso 8601 compliant date     object
dtype: object

Urgh! Our boolean col is an "object" lets add a check to fix it...

In [3]:
def fix_no_nulls_cols(column: pd.Series, fill_value=False) -> pd.Series:
    """Removes nulls from a column"""
    return column.fillna(fill_value).astype(bool)


validated = spicy_data.copy()
validated["no nulls allowed"] = validated["no nulls allowed"]

And "int between 0 and 1e6" is not an `int` type... because it has a `float` in it 🙃 

In [4]:
spicy_data["int between 0 and 1e6"]

0     934944.0
1     842425.0
2     220216.0
3     138980.0
4     907058.0
5     396576.0
6     870306.0
7     981140.0
8     266363.0
9     143593.0
10        -1.0
Name: int between 0 and 1e6, dtype: float64

### WAIT NO - it has a negative number in it!

Checking types didn't even show us that

In [5]:
validated["int between 0 and 1e6"] = (
    validated["int between 0 and 1e6"].astype(int).mask(lambda x: x <= 0, np.nan)
)

### Datetimes are great and never a problem!!

Alright it's fine... at least we can handle the dates right?

In [6]:
# all good!
validated["iso 8601 compliant date"] = pd.to_datetime(
    good_data["iso 8601 compliant date"]
)

Haha just kidding, we literally can't recover from the classic DD-MM or MM-DD ambiguity:

In [9]:
# pd.to_datetime(spicy_data["iso 8601 compliant date"])

P.S you can usually get around this by knowing some things about your dataset.


$ - $



# Okay *all* data is a mess, how do we fix it?

- You could deal with validation by...
- manually editing the raw data (bad!!)
- adding branching logic whenever you encounter an input data problem (pretty bad)
- adding ad-hoc checks in your code (better - but will create a mess)
- writing your own validation code that runs before your processing/analysis code (better)
- using an existing tool to do the validation for you (ideal)

# Roll your own validation schema

Please don't do this...

In [ ]:
# we need a dodgy function to do the validation with...
def validate_data(data: pd.DataFrame, schema: dict) -> pd.DataFrame:
    """Urgh this is a bit messy to debug if something is broken..."""
    for col, dtype in schema["dtypes"].items():
        if col not in data:
            raise ValueError(f"Missing column: {col}")
        if not all(isinstance(val, dtype) for val in data[col]):
            raise TypeError(f"Column '{col}' has incorrect data type")

        checks = schema["checks"].get(col, [])
        if checks:
            for check in checks:
                if not all(check(val) for val in data[col]):
                    raise ValueError(f"Column '{col}' failed validation checks")
    return data


# homebrew schema written as nested dict[dict]
my_dodgy_schema = {
    "dtypes": {
        "text": object,
        "int between 0 and 1e6": int,
        "float between -1 and 1": float,
        "no nulls allowed": bool,
    },
    "checks": {
        "text": None,
        "int between 0 and 1e6": [lambda x: 0 <= x <= 1e6],
        "float between -1 and 1": [lambda x: -1 <= x <= 1],
        "no nulls allowed": [lambda x: type(x) is bool],
    },
}

# check with known good
validated_good = good_data.copy()
# Validate good_data
try:
    validated_good = validate_data(good_data.copy(), my_dodgy_schema)
    print("This data is good so we should get to this statement...")
except (ValueError, TypeError) as e:
    print(f"Task failed succesfully with error: {e}")

# (Un?)Validate the bad data
try:
    validated_spicy = validate_data(spicy_data.copy(), my_dodgy_schema)
    print("Should NOT get to this statement!!")
except (ValueError, TypeError) as e:
    print(f"Task failed succesfully with error: {e}")

This data is good so we should get to this statement...
Task failed succesfully with error: Column 'int between 0 and 1e6' has incorrect data type


### Okay so that works - why is this bad?

- You, the overworked developer who dreams of playing video games tonight, have to maintain it.
- You have to test it to make sure it works.
- There are a lot of ways validation can go wrong and you have just discovered 0.0001% of those ways.

# Seeing the light: Strategies and Libraries

- Fear not young data tradie! We can stand on the shoulders of giants ❤️
- Other people have written tools to help us:
  - Pandera validation library
  - pola.rs lazy tables
  - `pd.DataFrame.astype()` and `pd.to_datetime()`.
  - Pydantic
  - python *schema*

# Pandera: you could probably get pretty far using this one

If you...
- are using pandas, pola.rs, dask, etc.
- like it when someone else sorted out a lot of the edge-cases for you
- deal with small, heterogenous datasets
- like clean and simple syntax

An example using the schema above:  
*Hey Gemini, re-write this custom schema as a Pandera DataFrameSchema:*  
*P.S I re-wrote some of this to show off the default check types which gemini is unaware of but translating schemas is bread-and-butter for LLMs*

In [11]:
import pandera as pa
from pandera.errors import SchemaError

# pandera comes with built in checkers we can use:
int_0_to_1e6 = pa.Check.in_range(0, 1e6)
float_minus_pos_1 = pa.Check.in_range(-1, 1)
# but we can always write our own with lambda or functions.
starts_with_row = pa.Check(lambda s: s.str.startswith("row"))

# this is how we define a pandera schema
my_pandera_schema = pa.DataFrameSchema(
    {
        "text": pa.Column(str, starts_with_row),
        "int between 0 and 1e6": pa.Column(int, checks=int_0_to_1e6),
        "float between -1 and 1": pa.Column(float, checks=float_minus_pos_1),
        "no nulls allowed": pa.Column(bool),
    }
)

good_pandera = my_pandera_schema(good_data)
print("The good dataset passed 😎")
try:
    spicy_pandera = my_pandera_schema(spicy_data)
    print("If we got here my talk is going poorly 🫢")
except SchemaError as e:
    print(f"The bad dataset failed with error: {e}")

The good dataset passed 😎
The bad dataset failed with error: Column 'text' failed element-wise validator number 0: <Check <lambda>> failure cases: 0


# Lazy Pola.rs 🐻‍❄️

- Polars does what Pandas does... but has the benefit of learning from the mistakes Pandas had to discover.
- Polars implements `lazy evaluation` through its Lazy API.
- This comes with some schema functionality!
- But it only covers typing and not validation 🥲

In [12]:
import polars as pl

# Pandas tools

While pandas doesn't have much validation out of the box it has some *very useful* functions:
- `pd.to_datetime`
- `pd.Series.astype`
- `pd.Series.fillna`
- etc.